# Project: Open street map
This project concentrate on the open street map database which is available for download, It is opensource and has XML format and reliable. We focuse on the city of chicago.

# Analysis:
As the first step we find names of the tags and their frequency. We save them in dictionary. For the big data we use the iterparse because we can not load all of the into the memory. Process of iterparse and parse are different. Also for loop looks different. Further informations are here.  
     
User guide of iterparse:  https://effbot.org/zone/element-iterparse.htm


One way of parsing is the event base parsing.  


ET.iterpase(filename, events=("start", )):

Event here is the start tag (<) it takes this block and take the event and elem of that which is the name of tag.


In [6]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import xml.etree.cElementTree as ET
import pprint

def count_tags(filename):
    tags = {}
    roots =  ET.iterparse(filename, parser = ET.XMLParser(encoding = 'iso-8859-5'))
#
    for event, elem in roots:
        if elem.tag in tags:
            tags[elem.tag] += 1
        else:
            tags[elem.tag] = 1

    return tags



def test():

    tags = count_tags('example.osm')
    pprint.pprint(tags)
    assert tags == {'bounds': 1,
                     'member': 3,
                     'nd': 4,
                     'node': 20,
                     'osm': 1,
                     'relation': 1,
                     'tag': 7,
                     'way': 1}

    

if __name__ == "__main__":
    test()

{'bounds': 1,
 'member': 3,
 'nd': 4,
 'node': 20,
 'osm': 1,
 'relation': 1,
 'tag': 7,
 'way': 1}



check the "k" value for each "<tag>" and see if they can be valid keys in MongoDB. Define different corrections as lower, lowercolon and ... and count them for "k" value.
we would like to change the data model and expand the "addr:street" type of keys to a dictionary like this: {"address": {"street": "Some value"}} So, we have to see if we have such tags, and if we have any tags with problematic characters.
    
    
When we look at the data tags flag tags the nodes.  Also there is tags which is describing the ways

In [15]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import xml.etree.ElementTree as ET
import pprint
import re


lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

def key_type(element, keys):
    if element.tag == "tag":
        
        if lower.search(element.attrib['k']):
            keys['lower'] += 1
        elif lower_colon.search(element.attrib['k']):
            keys['lower_colon'] += 1 
        elif problemchars.search(element.attrib['k']):
            keys['problemchars'] += 1
        else:
            keys['other'] += 1
        
    return keys



def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys



def test():
    # You can use another testfile 'map.osm' to look at your solution
    # Note that the assertions will be incorrect then.
    keys = process_map('example.osm')
    pprint.pprint(keys)
    assert keys == {'lower': 7, 'lower_colon': 0, 'other': 0, 'problemchars': 0}


if __name__ == "__main__":
    test()

{'lower': 7, 'lower_colon': 0, 'other': 0, 'problemchars': 0}


# Auditing stree Names:
The task in this exercise has two steps:
- audit the OSMFILE and change the variable 'mapping' to reflect the changes needed to fix 
    the unexpected street types to the appropriate ones in the expected list.
    You have to add mappings only for the actual problems you find in this OSMFILE,
    not a generalized solution, since that may and will depend on the particular area you are auditing.
- write the update_name function, to actually fix the street name.
    The function takes a string with street name as an argument and should return the fixed name
    We have provided a simple test so that you see what exactly is expected


In [26]:

import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

OSMFILE = "examplenum2.osm"
# it matches the very last work in a street
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

# These are normal names which we dont want to see them in output 
expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons"]

# UPDATE THIS VARIABLE
mapping = { "St": "Street",
            "St.": "Street",
            "Ave": "Avenue",
            "Rd.": "Road"
            }

# it has regular expression type.
def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)

# WE check the attribute of that element and we want it has this values "addr:street"
def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    # elem is an object (dictionary) that have  star tags such as nodes, relations... 
    # events is the start tag (<) it takes this block and take the event and elem of that which is the name of tag
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        # way is the name of street
        if elem.tag == "node" or elem.tag == "way":
            # iter method make an iteration and return all of the subtags. The tags nested in this element.
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])

    return street_types


def update_name(name, mapping):

    m = street_type_re.search(name)
# it means that if this conditions it true
    if m:
        street_type = m.group()
# exclude the normal names in the expected list. If the name is not in the list add it to the unusual list
        if street_type not in expected:
            name = re.sub(street_type_re, mapping[street_type], name)
    return name


def test():
    st_types = audit(OSMFILE)
    assert len(st_types) == 3
    pprint.pprint(dict(st_types))

    for st_type, ways in st_types.items():
        for name in ways:
            better_name = update_name(name, mapping)
            print(name, "=>", better_name)
            if name == "West Lexington St.":
                assert better_name == "West Lexington Street"
            if name == "Baldwin Rd.":
                assert better_name == "Baldwin Road"


if __name__ == '__main__':
    test()

{'Ave': {'North Lincoln Ave', 'N. Lincoln Ave'},
 'Rd.': {'Baldwin Rd.'},
 'St.': {'West Lexington St.'}}
North Lincoln Ave => North Lincoln Avenue
N. Lincoln Ave => N. Lincoln Avenue
Baldwin Rd. => Baldwin Road
West Lexington St. => West Lexington Street
